In [1]:
import numpy as np
import pandas as pd

# Importing data
health = pd.read_csv("../Datasets/rankmd.csv", delimiter=";")
FastFood = pd.read_csv("../Datasets/FastFoodRestaurants.csv")
income = pd.read_csv("../Datasets/kaggle_income.csv", encoding="ISO 8859-1")
poldata = pd.read_csv("../Datasets/2020_US_County_Level_Presidential_Results.csv", delimiter=",")

us_state_to_abbrev = {
    "Alabama": "AL",
    "Alaska": "AK",
    "Arizona": "AZ",
    "Arkansas": "AR",
    "California": "CA",
    "Colorado": "CO",
    "Connecticut": "CT",
    "Delaware": "DE",
    "Florida": "FL",
    "Georgia": "GA",
    "Hawaii": "HI",
    "Idaho": "ID",
    "Illinois": "IL",
    "Indiana": "IN",
    "Iowa": "IA",
    "Kansas": "KS",
    "Kentucky": "KY",
    "Louisiana": "LA",
    "Maine": "ME",
    "Maryland": "MD",
    "Massachusetts": "MA",
    "Michigan": "MI",
    "Minnesota": "MN",
    "Mississippi": "MS",
    "Missouri": "MO",
    "Montana": "MT",
    "Nebraska": "NE",
    "Nevada": "NV",
    "New Hampshire": "NH",
    "New Jersey": "NJ",
    "New Mexico": "NM",
    "New York": "NY",
    "North Carolina": "NC",
    "North Dakota": "ND",
    "Ohio": "OH",
    "Oklahoma": "OK",
    "Oregon": "OR",
    "Pennsylvania": "PA",
    "Rhode Island": "RI",
    "South Carolina": "SC",
    "South Dakota": "SD",
    "Tennessee": "TN",
    "Texas": "TX",
    "Utah": "UT",
    "Vermont": "VT",
    "Virginia": "VA",
    "Washington": "WA",
    "West Virginia": "WV",
    "Wisconsin": "WI",
    "Wyoming": "WY",
    "District of Columbia": "DC",
    "American Samoa": "AS",
    "Guam": "GU",
    "Northern Mariana Islands": "MP",
    "Puerto Rico": "PR",
    "United States Minor Outlying Islands": "UM",
    "U.S. Virgin Islands": "VI",
}
    
# Inverting the dictionary
abbrev_to_us_state = dict(map(reversed, us_state_to_abbrev.items()))

# Creating a state dataset
FastFood['State'] = FastFood['province'].map(abbrev_to_us_state)
States = health.copy()
States = States[States['FIPS'].astype(str).str.endswith('000')]

# Converting FIPS to string
health['FIPS']=health['FIPS'].apply(lambda x: '{0:0>5}'.format(x))
health["food_environment_index_Food Environment Index"] = health["food_environment_index_Food Environment Index"].str.replace(",",".").astype(float)

In [2]:
# Removing ' County' from the county names in income and political data
income["County"] = income.County.str.replace(' County', '')
poldata["county_name"] = poldata.county_name.str.replace(' County', '')

# Merging income and health data
temp_df = income.groupby(["State_Name","County"]).mean().reset_index()
new_df = pd.merge(health.copy(), temp_df.copy(),  how='left', left_on=['State','County'], right_on = ['State_Name','County'])

In [3]:
#ONLY NEEDS TO BE RAN ONCE AS THE COUNTIES ARE STORED IN THE CSV.

#!{sys.executable} -m pip install pgeocode
#import pgeocode

#nomi = pgeocode.Nominatim('us')
#county_names = []
#for i in range(len(FastFood)):
#    county_names.append(nomi.query_postal_code(FastFood["postalCode"][i]).county_name)
    
#FastFood["County"] = county_names
#FastFood.to_csv("../Datasets/FastFoodRestaurants.csv")

In [4]:
# Merging fastfood data with income and health data
temptemp = FastFood.groupby(["State", "County"]).count().reset_index()[['State','County','address']]
tempo = temptemp.rename(columns={'address':'nr of FFchains'})
data_df = pd.merge(new_df, tempo,  how='left', left_on=['State','County'], right_on =['State','County'])
data_df['nr of FFchains'] = data_df['nr of FFchains'].fillna(0)

# Merging with the political data 
merged=pd.merge(data_df.copy(), poldata.copy(),  how='left', left_on=['State','County'], right_on = ['state_name','county_name'])

# Snipping the columns to a more clean dataset
data = merged[["premature_deathYears_of_Potential_Life_Lost_Rate",'adult_obesity_% Adults with Obesity',
                "adult_smoking_% Smokers", "excessive_drinking_% Excessive Drinking", "food_environment_index_Food Environment Index",
                "uninsured_% Uninsured", "unemployed_% Unemployed", 'nr of FFchains', 'Mean',
                "poor_physical_health_days_Average Number of Physically Unhealthy Days",
                "poor_mental_health_days_Average Number of Mentally Unhealthy Days","per_gop"]]

# Dropping NaNs
data = data.dropna()

# Creating a response value to predict a ML model
data['is_obese'] = data['adult_obesity_% Adults with Obesity']>=33
data = data.drop(['adult_obesity_% Adults with Obesity'],axis=1)

# Cleansing columns, making unemployed percentage an integer and physical and mental unhealthy days floats
data["unemployed_% Unemployed"] = data["unemployed_% Unemployed"].str.replace(",",".").astype(float).astype(int)
data["poor_physical_health_days_Average Number of Physically Unhealthy Days"] = data["poor_physical_health_days_Average Number of Physically Unhealthy Days"].str.replace(",",".").astype(float)
data["poor_mental_health_days_Average Number of Mentally Unhealthy Days"] = data["poor_mental_health_days_Average Number of Mentally Unhealthy Days"].str.replace(",",".").astype(float)


In [5]:
# Storing the merged, cleansed datasets as csv files
data.to_csv("../Datasets/Mixed_data.csv")
merged.to_csv("../Datasets/All_data.csv")

In [6]:
data

,premature_deathYears_of_Potential_Life_Lost_Rate,adult_smoking_% Smokers,excessive_drinking_% Excessive Drinking,food_environment_index_Food Environment Index,uninsured_% Uninsured,unemployed_% Unemployed,nr of FFchains,Mean,poor_physical_health_days_Average Number of Physically Unhealthy Days,poor_mental_health_days_Average Number of Mentally Unhealthy Days,per_gop,is_obese
1,7830.0,20,14,6.7,10.0,2,0.0,53735.557235,4.5,4.9,0.714368,True
3,11477.0,26,12,5.5,14.0,3,1.0,37725.000000,5.6,5.6,0.534512,True
5,11096.0,23,16,7.9,14.0,2,3.0,55127.000000,5.0,5.4,0.895716,True
7,12848.0,23,13,6.8,12.0,3,0.0,27993.000000,5.4,5.7,0.575253,True
9,10371.0,23,14,6.7,12.0,2,4.0,45107.000000,5.0,5.5,0.572690,True
...,...,...,...,...,...,...,...,...,...,...,...,...
3185,8616.0,20,21,7.6,12.0,3,2.0,127999.000000,3.7,4.1,0.791794,False
3186,6136.0,18,21,7.6,11.0,3,3.0,68733.000000,3.3,3.8,0.720903,False
3188,7775.0,19,19,7.5,11.0,3,2.0,0.000000,3.6,4.0,0.736554,False
3190,8081.0,21,18,7.3,12.0,3,2.0,89130.000000,4.0,4.2,0.797277,True
